In [1]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '20'
os.environ['MKL_NUM_THREADS'] = '20'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import datetime as dt
import copy
import statsmodels.api as sm
from matplotlib.colors import LogNorm
import xarray as xr
import pickle
import random
import time
import signal
import multiprocessing
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from scipy import interpolate
import random
import lightgbm as lgb
import itertools
from scipy import spatial
import joblib



def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import sys
sys.path.insert(1,'/home/jsperezc/jupyter/AQ_Forecast/functions/')
import postprocessing
from chained_models import *

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import RegressorChain, MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import Ridge

from denseweight import DenseWeight
# from gbdtmo import GBDTMulti, load_lib

In [2]:
coor_esta= pd.read_csv("/var/data1/AQ_Forecast_DATA/historic/PM25/CoordenadasEstaciones.csv", index_col= "Nombre")


Estaciones = ["ITA-CJUS", "CAL-LASA", "ITA-CONC", "MED-LAYE", "CAL-JOAR", "EST-HOSP", "MED-ALTA", "MED-VILL", "BAR-TORR", 
              "COP-CVID", "MED-BEME", "MED-TESO", "MED-SCRI", "MED-ARAN", "BEL-FEVE", "ENV-HOSP", "SAB-RAME", "MED-SELE",
             "CEN-TRAF","SUR-TRAF"]


#coor_esta = coor_esta.drop(short_stations,axis = 0)
# coor_esta = coor_esta.loc[Estaciones]

file_gfs_ref = os.listdir("/var/data1/AQ_Forecast_DATA/historic/GFS/historic/00/")[0]
gfs_referencia = xr.open_dataset("/var/data1/AQ_Forecast_DATA/historic/GFS/historic/00/" + file_gfs_ref)  
gfs_referencia = postprocessing.recorte_espacial(gfs_referencia)

max_lag = 96 ## Maximum lag used
n_leadtime = 96
keys_gfs = ['tcc','rad','prate','hpbl','cin']

def clean_abrupt_gradient(anoms):
    diff = abs(np.diff(anoms))
    anoms[np.hstack([diff>=1.5,np.array([False])])] = np.nan
    anoms[np.hstack([np.array([False]),diff>=1.5])] = np.nan
    return pd.Series(anoms,index = anoms.index)

def get_optimal_window(variable,hour):
    return np.argmax(abs(np.load('/var/data1/AQ_Forecast_DATA/historic/GFS/correlations/CorrsAnomsGFS_'\
                                 +variable+'_v3.npy')[:,hour]))+1


In [3]:
class CustomMSELoss:
    def __call__(self, y_true, y_pred):
        grad = 2 * (y_pred - y_true)
        hess = 2 * np.ones_like(y_pred)
        return grad, hess

class CustomWeightedMSELoss:
    def __init__(self, weighting_function):
        self.weighting_function = weighting_function
        
    def __call__(self, y_true, y_pred):
        # Compute the weighted differences
        weighted_diff = self.weighting_function(y_true) * (y_pred - y_true)
        # Compute the weighted MSE loss
        grad = 2 * weighted_diff
        hess = 2 * self.weighting_function(y_true)
        
        return grad, hess

In [4]:
### CAMS

data_CAMS = postprocessing.call_files('CAMS')
data_CAMS = postprocessing.recorte_espacial(data_CAMS)

dataset_full = data_CAMS

lat_cams = dataset_full.latitude.values
lon_cams = dataset_full.longitude.values

aod = np.nanmean(dataset_full.aod550.values,axis = (1,2))#[:].data
bcaod = np.nanmean(dataset_full.bcaod550.values,axis = (1,2))#[:].data
pm2p5_cams = np.nanmean(dataset_full.pm2p5.values,axis = (1,2)) #* 1000_000_000

df_cams = pd.DataFrame(np.array([aod,bcaod,pm2p5_cams]).T,index= dataset_full.time.values,
    columns = ['aod','bcaod','pm2p5_cams'])
# df_cams = pd.DataFrame(np.array([aod,pm2p5_cams]).T,index= dataset_full.time.values,
#     columns = ['aod','pm2p5_cams'])
# df_cams = (df_cams.rolling(3,center=True).mean())
df_cams.index = df_cams.index - dt.timedelta(hours = 5)


In [5]:
# df_cams.to_csv('/var/data1/AQ_Forecast_DATA/zenodo/df_cams.csv')

In [6]:
data_pm25 = pd.read_csv('/var/data1/AQ_Forecast_DATA/historic/PM25/FullPM2.5.csv',index_col=0,parse_dates=True)
short_stations = ['SAB-JOFE','MED-UNNV','GIR-TEMP','GIR-SOSN','GIR-EPM','MED-FISC']
# data_pm25.index = data_pm25.index-dt.timedelta(hours = 1) 
data_pm25 = data_pm25.drop(short_stations,axis = 1)
stations = np.sort(data_pm25.keys())
data_pm25 = data_pm25['2017-10-01':'2020-04-01']
data_pm25 = data_pm25.interpolate(limit=3)

In [7]:
ifrp = pd.read_csv("/var/data1/AQ_Forecast_DATA/historic/Fires/Processed/IFRP_Radios_800hpa_4dias.csv", 
                   usecols=[str(i) for i in np.arange(16)],index_col=0,parse_dates=True)
ifrp.columns = np.arange(50, 800, 50).astype(str)

ifrp_optimal_50 = ifrp[['50']].rolling(24).mean()

In [8]:
base_estimators = {
    'gb_ch':
         MultiOutputLGBM(**{
            'task': 'train', 
            'num_leaves': 5,
            'min_child_samples':150,
            'max_depth':30,
            'extra_trees':'true',
            'learning_rate':0.05,
            'n_jobs': 20,
            'linear_tree': True,
            'verbose': -1}),
    'gb_mo':
         MultiOutputRegressor(lgb.LGBMRegressor(**{'task': 'train',
            'num_leaves': 5,
            'min_child_samples': 150,
            'max_depth':30,
            'extra_trees': 'true',
            'learning_rate':0.05,
            'n_jobs': 1,                               
            'verbose': -1,
            'importance_type': 'gain',
            'linear_tree': True,
            }), n_jobs=20),
    
    'rf_ch':
         MultiOutputRF(**{'bootstrap': True,
             'max_samples': 0.2,
             'max_depth': 10,
             'max_features': 10,
             'n_estimators': 50,
             'n_jobs': 20}),
    'rf_mo':
         MultiOutputRegressor(RandomForestRegressor(**{'bootstrap': True,
         'max_samples': 0.4,
         'max_depth': 10,
         'max_features': 80,
         'n_estimators': 50,
         'n_jobs': 1}), n_jobs=20),
    
    'lr_rc':
         MultiOutputLR(**{'n_jobs': 1}),
    
    'lr_mo':
         MultiOutputRegressor(Ridge(), n_jobs=1)}

In [9]:
import dill

def train_estimator(X,Y,estimator,path_scaler_x=None,path_scaler_y=None,path_estimator=None,weighted_loss=False):
    #Scale
    scaler_x = RobustScaler().fit(X)
    scaler_y = RobustScaler().fit(Y)
    X_scaled = scaler_x.transform(X)
    Y_scaled = scaler_y.transform(Y) 
    
    if path_scaler_x!=None:
        joblib.dump(scaler_x, open(path_scaler_x, 'wb'))
    if path_scaler_y!=None:
        joblib.dump(scaler_y, open(path_scaler_y, 'wb'))
#     print(X_scaled)
    if weighted_loss==True:
        # Define DenseWeight
        dw = DenseWeight(alpha=1.15)
        weights_temp = dw.fit(Y_scaled[:,0])
        print(path_estimator)
        if '_MO' in path_estimator:
            estimator.estimator.set_params(objective=CustomWeightedMSELoss(dw))
        elif '_CH' in path_estimator:
            estimator.kwargs['objective'] = CustomWeightedMSELoss(dw)
    estimator.fit(X_scaled,Y_scaled)
    
    if path_estimator!=None:
        with open(path_estimator, 'wb') as f:
            dill.dump(estimator, f)
        
    return estimator



In [10]:
def cross_validate_estimator(X,Y,estimator,n_splits = 5,gap=96,weighted_loss=False,name_estimator = ''):
    #Scale
    scaler_x = RobustScaler().fit(X)
    scaler_y = RobustScaler().fit(Y)
    X_scaled = scaler_x.transform(X)
    Y_scaled = scaler_y.transform(Y) 
    
    #Get splits
    tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)
    
    mses=[]
    for train_cv, test_cv in tscv.split(X=X_scaled):
        X_train_temp = X_scaled[train_cv]
        Y_train_temp = Y_scaled[train_cv]

        X_test_temp = X_scaled[test_cv]
        Y_test_temp = Y_scaled[test_cv]
        if weighted_loss==True:
            # Define DenseWeight
            dw = DenseWeight(alpha=1.15)
            weights_temp = dw.fit(Y_scaled[:,0])
            if '_MO' in name_estimator:
                estimator.estimator.set_params(objective=CustomWeightedMSELoss(dw))
            elif '_CH' in name_estimator:
                estimator.kwargs['objective'] = CustomWeightedMSELoss(dw)
        estimator.fit(X_train_temp,Y_train_temp)
        y_model = estimator.predict(X_test_temp)
#         y_model_full = estimator.predict_random(X_test_temp)
#         for j in range(y_model_full.shape[0]):
#             y_model_full[j] = scaler_y.inverse_transform(y_model_full[j])
        y_model = scaler_y.inverse_transform(y_model)
        y_val = scaler_y.inverse_transform(Y_test_temp)
    
        mses.append(mean_squared_error(y_model,y_val))
#     return y_val,y_model, y_model_full, np.mean(mses)
    return np.mean(mses)

In [11]:
### To validation errors dataframe:

station_names = []
estimator_names = []
cv_error = []

for station_i in Estaciones[:]:
# for station_i in ['ENV-HOSP']:
    print(station_i)
    df_mean = pd.DataFrame(data_pm25[station_i])
    df_mean.columns = ["PM25_stations"]
    
    ## Definición de la serie de tiempo de cams para cada estación (un sólo pixel)
    lat_cams = dataset_full.latitude.values
    lon_cams = dataset_full.longitude.values
    
    ## Station coordinates
    lat_est = coor_esta.loc[station_i].Latitud
    lon_est = coor_esta.loc[station_i].Longitud
    
    df_hourly_chem = pd.concat([df_mean,df_cams],axis=1).dropna()
    ###Read GFS###################
    
    latitudes, longitudes = np.array(lat_est), np.array(lon_est)
    xs_gfs, ys_gfs = np.meshgrid(gfs_referencia.longitude, gfs_referencia.latitude)
    arbol_binario = spatial.cKDTree(np.c_[xs_gfs.ravel(), ys_gfs.ravel()])
    
    lxs, lys = np.meshgrid(longitudes, latitudes)
    distancia, indice = arbol_binario.query([longitudes, latitudes])
        
    lon_file = np.c_[xs_gfs.ravel(), ys_gfs.ravel()][indice][0]
    lat_file = np.c_[xs_gfs.ravel(), ys_gfs.ravel()][indice][1]
    
    df_GFS_hourly = pd.read_csv(f'/var/data1/AQ_Forecast_DATA/historic/GFS/pixel_series/GFS_{lat_file}_{lon_file}.csv',index_col = 0, parse_dates=True)
    df_GFS_hourly.index = df_GFS_hourly.index - dt.timedelta(hours = 1)
    
    ### PREPARA DATOS DE ENTRADA

    df_optimal_cams = pd.DataFrame(index = df_hourly_chem.index)
    df_optimal_cams['aod'] = df_hourly_chem.aod.rolling(3).mean()
    df_optimal_cams['bcaod'] = df_hourly_chem.bcaod.rolling(1).mean()
    df_optimal_cams['pm2p5_cams'] = df_hourly_chem.pm2p5_cams.rolling(3).mean()

    df_optimal_gfs = pd.DataFrame(index = df_GFS_hourly.index)

    for variable in keys_gfs:
        df_optimal_temp = copy.deepcopy(df_GFS_hourly[[variable]])
        for hour in range(0,24):
            optimal_window = get_optimal_window(variable,hour)
            df_optimal_temp[df_optimal_temp.index.hour == hour] = df_GFS_hourly[[variable]].rolling(optimal_window).mean()\
                [df_optimal_temp.index.hour == hour]
        df_optimal_gfs[variable] = df_optimal_temp

    df_hour = pd.DataFrame(df_hourly_chem.index.hour, index = df_hourly_chem.index)
    df_hour1 = np.sin(2*np.pi*(df_hour/24))
    df_hour2 = np.cos(2*np.pi*(df_hour/24))

    df_dow = pd.DataFrame(df_hourly_chem.index.dayofweek, index = df_hourly_chem.index)
    df_dow1 = np.sin(2*np.pi*(df_dow/7))
    df_dow2 = np.cos(2*np.pi*(df_dow/7))
    
    max_lag = 48
    x_shifts = pd.concat([df_hourly_chem.PM25_stations.shift(i) for i in range(max_lag,0,-1)],axis = 1).dropna()
    x_shifts.columns = np.array(['PM_'+str(i).zfill(2) for i in range(max_lag,0,-1)])

    aod_future = pd.DataFrame(np.array([df_optimal_cams[['aod']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_cams[['aod']].index,\
                 columns = np.array(['AOD_'+str(i).zfill(2) for i in range(0,97,3)]))
    bc_future = pd.DataFrame(np.array([df_optimal_cams[['bcaod']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_cams[['bcaod']].index,\
                 columns = np.array(['BCAOD_'+str(i).zfill(2) for i in range(0,97,3)]))
    pm2p5_future = bc_future = pd.DataFrame(np.array([df_optimal_cams[['pm2p5_cams']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_cams[['pm2p5_cams']].index,\
                 columns = np.array(['PMCAMS_'+str(i).zfill(2) for i in range(0,97,3)]))

    tcc_future = pd.DataFrame(np.array([df_optimal_gfs[['tcc']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_gfs[['tcc']].index,\
                 columns = np.array(['tcc_'+str(i).zfill(2) for i in range(0,97,3)]))
    rad_future = pd.DataFrame(np.array([df_optimal_gfs[['rad']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_gfs[['rad']].index,\
                 columns = np.array(['rad_'+str(i).zfill(2) for i in range(0,97,3)]))
    prate_future = pd.DataFrame(np.array([df_optimal_gfs[['prate']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_gfs[['prate']].index,\
                 columns = np.array(['prate_'+str(i).zfill(2) for i in range(0,97,3)]))
    hpbl_future = pd.DataFrame(np.array([df_optimal_gfs[['hpbl']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_gfs[['hpbl']].index,\
                 columns = np.array(['hpbl_'+str(i).zfill(2) for i in range(0,97,3)]))
    cin_future = pd.DataFrame(np.array([df_optimal_gfs[['cin']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = df_optimal_gfs[['cin']].index,\
                 columns = np.array(['cin_'+str(i).zfill(2) for i in range(0,97,3)]))


    ifrp50_future = pd.DataFrame(np.array([ifrp_optimal_50[['50']].shift(-i).values for i in range(0,97,3)])[:,:,0].T,\
                 index = ifrp_optimal_50[['50']].index,\
                 columns = np.array(['ifrp50_'+str(i).zfill(2) for i in range(0,97,3)]))

    dow1_future = pd.DataFrame(np.array([df_dow1[[0]].shift(-i).values for i in range(0,96,24)])[:,:,0].T,\
                 index = df_dow1[[0]].index,\
                 columns = np.array(['dow1_'+str(i).zfill(2) for i in range(0,96,24)]))
    dow2_future = pd.DataFrame(np.array([df_dow2[[0]].shift(-i).values for i in range(0,96,24)])[:,:,0].T,\
                 index = df_dow2[[0]].index,\
                 columns = np.array(['dow2_'+str(i).zfill(2) for i in range(0,96,24)]))

    hod1_future = pd.DataFrame(np.array([df_hour1[[0]].shift(-i).values for i in range(0,24,1)])[:,:,0].T,\
                 index = df_hour1[[0]].index,\
                 columns = np.array(['hod1_'+str(i).zfill(2) for i in range(0,24,1)]))
    hod2_future = pd.DataFrame(np.array([df_hour2[[0]].shift(-i).values for i in range(0,24,1)])[:,:,0].T,\
                 index = df_hour2[[0]].index,\
                 columns = np.array(['hod2_'+str(i).zfill(2) for i in range(0,24,1)]))

    y_future = pd.DataFrame(np.array([df_hourly_chem[['PM25_stations']].shift(-i).values for i in range(0,96)])[:,:,0].T,\
                 index = df_hourly_chem[['PM25_stations']].index,\
                 columns = np.array(['y_'+str(i).zfill(2) for i in range(0,96)]))
    
    ### try rolling mean output # this was a winner
    pm_smoothed = df_hourly_chem[['PM25_stations']].rolling(26,min_periods=16).mean()
    y_future_smoothed = pd.DataFrame(np.array([pm_smoothed.shift(-i).values for i in range(0,96)])[:,:,0].T,\
                 index = pm_smoothed.index,\
                 columns = np.array(['y_'+str(i).zfill(2) for i in range(0,96)]))
    
    X = pd.concat([x_shifts,
    aod_future,
#     bc_future, NO USAR NUNCA, DAÑAN EL PRONÓSTICO :C
#     pm2p5_future,
    tcc_future,
#     rad_future, meh
    prate_future,
    hpbl_future,
#     cin_future, meh
    ifrp50_future,
    dow1_future,
    dow2_future,
    hod1_future,
    hod2_future],axis=1).dropna()

    Y = y_future_smoothed.loc[X.index] 
    XY = pd.concat([X,Y],axis=1).dropna()
    X = XY.iloc[:,:-96]
    Y = XY.iloc[:,-96:]
#     for estimator_name in base_estimators.keys():
    for estimator_name in ['gb_mo','gb_ch']:    
#         try:
        if ('gb_mo' in estimator_name)|('gb_ch' in estimator_name):
            estimator_temp = copy.deepcopy(base_estimators[estimator_name])
            train_estimator(X,Y,estimator_temp,
             '/var/data1/AQ_Forecast_DATA/scalers_v3/'+station_i+'_X.scl',
             '/var/data1/AQ_Forecast_DATA/scalers_v3/'+station_i+'_Y.scl',
             '/var/data1/AQ_Forecast_DATA/trained_estimators_v3/'+station_i+'_'+str(estimator_name.upper())+'.mdl',weighted_loss=True)
#                 error_temp = np.sqrt(cross_validate_estimator(X,Y, estimator_temp,weighted_loss=True,name_estimator=estimator_name))
#         #         aaaaa
#                 station_names.append(station_i)
#                 estimator_names.append(estimator_name)
#                 cv_error.append(error_temp)
#                 print(estimator_name,str(error_temp))
        else:
            estimator_temp = copy.deepcopy(base_estimators[estimator_name])
            train_estimator(X,Y,estimator_temp,
             '/var/data1/AQ_Forecast_DATA/scalers_v3/'+station_i+'_X.scl',
             '/var/data1/AQ_Forecast_DATA/scalers_v3/'+station_i+'_Y.scl',
             '/var/data1/AQ_Forecast_DATA/trained_estimators_v3/'+station_i+'_'+str(estimator_name.upper())+'.mdl',weighted_loss=False)
#                 error_temp = np.sqrt(cross_validate_estimator(X,Y, estimator_temp,weighted_loss=False,name_estimator=estimator_name))
#         #         aaaaa
#                 station_names.append(station_i)
#                 estimator_names.append(estimator_name)
#                 cv_error.append(error_temp)
#                 print(estimator_name,str(error_temp))

#         except Exception as e: 
#             print(e)
#             station_names.append(station_i)
#             estimator_names.append(estimator_name)
#             cv_error.append(np.nan)
#         aaaa    
#         df_errors = pd.DataFrame(columns = ['station','estimator','error'])    
#         df_errors['station'] = station_names
#         df_errors['estimator'] = estimator_names
#         df_errors['error'] = cv_error
#         df_errors.to_csv('/var/data1/AQ_Forecast_DATA/errors/cv_stations_v3.csv')  
#         aaaa
        

ITA-CJUS
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/ITA-CJUS_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/ITA-CJUS_GB_CH.mdl
CAL-LASA
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/CAL-LASA_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/CAL-LASA_GB_CH.mdl
ITA-CONC
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/ITA-CONC_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/ITA-CONC_GB_CH.mdl
MED-LAYE
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/MED-LAYE_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/MED-LAYE_GB_CH.mdl
CAL-JOAR
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/CAL-JOAR_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/CAL-JOAR_GB_CH.mdl
EST-HOSP
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/EST-HOSP_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/EST-HOSP_GB_CH.mdl
MED-ALTA
/var/data1/AQ_Forecast_DATA/trained_estimators_v3/MED-ALTA_GB_MO.mdl
/var/data1/AQ_Forecast_DATA/trained_esti